In [ ]:
import torch
from torch import nn, optim
import mltools
import dataloader as dl

In [ ]:
device = torch.device("cuda")
train_iter, val_iter, test_iter = dl.mnist(batch_size=1000)  # 获取训练集、验证集、测试集
model = nn.Sequential(nn.Flatten(), nn.Linear(784, 256), nn.ReLU(), nn.Linear(256, 10))  # 设置模型结构
model.to(device)
loss = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=1e-1)
ml = mltools.MachineLearning("mlp")
ml.add_model(model)
epoch, timer, recorder = ml.batch_create()

In [ ]:
# 训练模型
num_epochs = epoch(20)
animator = ml.create_animator(xlabel="epoch", xlim=[0, epoch.totol_epoch + 1], ylim=-0.1, legend=["train loss", "train acc", "val acc"])  # 创建动画器
for current_epoch in range(1, num_epochs + 1):
    timer.start()  # 开始计时

    # 计算训练集
    metric_train = mltools.Accumulator(3)  # 累加器：(train_loss, train_acc, train_size)
    model.train()  # 训练模式
    for x, y in train_iter:
        x, y = x.to(device), y.to(device)  # 转换x、y
        y_train = model(x)  # 计算模型
        train_loss = loss(y_train, y)  # 计算损失
        train_pred = y_train.argmax(dim=1)  # 计算预测值
        train_acc = (train_pred == y).sum()  # 计算准确数

        # 梯度更新
        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()

        metric_train.add(train_loss * y.numel(), train_acc, y.numel())  # 累加损失、准确数、样本数
    recorder[0].append(metric_train[0] / metric_train[2])  # 计算平均损失
    recorder[1].append(metric_train[1] / metric_train[2])  # 计算准确率

    # 计算验证集
    metric_val = mltools.Accumulator(2)  # 累加器：(val_loss, val_size)
    model.eval()  # 验证模式
    with torch.no_grad():
        for x, y in val_iter:
            x, y = x.to(device), y.to(device)  # 转换x、y
            y_val = model(x)  # 计算模型
            val_pred = y_val.argmax(dim=1)  # 计算预测值
            val_acc = (val_pred == y).sum()  # 计算准确数
            metric_val.add(val_acc, y.numel())  # 累加准确数、样本数
    recorder[2].append(metric_val[0] / metric_val[1])  # 计算准确率

    timer.stop()  # 停止计时

    # 打印输出值
    ml.logger.info(f"train loss {recorder[0][-1]:.3f}, train acc {recorder[1][-1]:.3f}, val acc {recorder[2][-1]:.3f}")
    ml.print_training_time_massage(timer, num_epochs, current_epoch)
    ml.logger.info(f"trained on {str(device)}")
    animator.show(recorder.data)
else:
    # 打印输出值
    ml.logger.info(f"train loss {recorder[0][-1]:.3f}, train acc {recorder[1][-1]:.3f}, val acc {recorder[2][-1]:.3f}")
    ml.print_training_time_massage(timer, num_epochs, current_epoch)
    ml.logger.info(f"trained on {str(device)}")
    animator.show(recorder.data)
ml.save()

In [ ]:
# 测试模型
model.eval()
metric = mltools.Accumulator(2)  # 累加器：(test_acc, test_size)
with torch.no_grad():
    for x, y in test_iter:
        x, y = x.to(device), y.to(device)  # 转换x、y
        y_test = model(x)  # 计算模型
        test_pred = y_test.argmax(dim=1)  # 计算预测值
        test_acc = (test_pred == y).sum()  # 计算准确数
        metric.add(test_acc, y.numel()) # 累加准确数、样本数
ml.logger.info(f"test acc {metric[0] / metric[1]:.3f}")  # 计算准确率并输出

In [ ]:
# 预测模型
model.eval()
x, y = next(iter(test_iter))  # 从测试中取一个批量
x, y = x[:10].to(device), y[:10].to(device) # 转换x、y
y_pred = model(x)  # 计算模型
y_pred = y_pred.argmax(dim=1)  # 计算预测值
labels = [f"real:{y[index]}\npred:{y_pred[index]}" for index in range(y.numel())]
mltools.images(x.squeeze(1), labels, shape=(2, 5))